### The file can be used to understand the use of a widget in a holoviews plot.

This code was later used to add widgets to the interactive map where we change NFFT to see how it affects spectrograms.

In [ ]:
import param
import numpy as np
import holoviews as hv
from holoviews import opts

hv.extension('bokeh', 'matplotlib')

In [ ]:
from scipy.misc import ascent

In [ ]:
print(ascent().shape)
temp = ascent()

In [ ]:
temp[::2, ::2].shape

In [ ]:
stairs_image = hv.Image(ascent()[:, :], label="stairs")
stairs_image

In [ ]:
from scipy import interpolate

x = np.arange(0,512,8)
y = np.arange(0,512,8)

xx, yy = np.meshgrid(x, y)

z = ascent()[::8, ::8]

f = interpolate.interp2d(x, y, z, kind='cubic')
x2 = np.arange(0,512,1)
y2 = np.arange(0,512,1)

newz = f(x2, y2)

print(newz.shape)
stairs_image = hv.Image(newz, label="stairs")
stairs_image

In [ ]:
# doing subsampling 
info = 9
from scipy import ndimage
from scipy import interpolate

class image_filter(hv.Operation):
    
    sigma = param.Number(default=2)

    def _process(self, element, key=None):
        x = np.arange(0,512,self.p.sigma)
        y = np.arange(0,512,self.p.sigma)
        
        xx, yy = np.meshgrid(x, y)
        
        z = ascent()[::self.p.sigma, ::self.p.sigma]
        f = interpolate.interp2d(x, y, z, kind='cubic')
        
        x2 = np.arange(0,512,1)
        y2 = np.arange(0,512,1)
        
        newz = f(x2, y2)
        return hv.Image(newz, label="stairs")

stairs_map = hv.HoloMap({sigma: image_filter(stairs_image,sigma=sigma)
                         for sigma in range(1, 12, 1)}, kdims="Sigma")

stairs_map.opts(framewise=True)

In [ ]:
stairs_image = hv.Image(ascent()[200:500, :], bounds=[0, 0, ascent().shape[1], 300], label="stairs")
stairs_image

In [ ]:
from scipy import ndimage

class image_filter(hv.Operation):
    
    sigma = param.Number(default=10)
    
    type_ = param.String(default="low-pass")

    def _process(self, element, key=None):
        xs = element.dimension_values(0, expanded=False)
        ys = element.dimension_values(1, expanded=False)
        
        # setting flat=False will preserve the matrix shape
        data = element.dimension_values(2, flat=False)
        
        new_data = ndimage.gaussian_filter(data, self.p.sigma)
        
        # make an exact copy of the element with all settings, just with different data and label:
        element = element.clone((xs, ys, new_data))
        return element

    
stairs_map = hv.HoloMap({sigma: image_filter(stairs_image, sigma=sigma)
                         for sigma in range(0, 12, 1)}, kdims="Sigmakdsmf")

stairs_map.opts(framewise=True)

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)

dt = 0.0005
t = np.arange(0.0, 20.0, dt)
s1 = np.sin(2 * np.pi * 100 * t)
s2 = 2 * np.sin(2 * np.pi * 400 * t)

# create a transient "chirp"
s2[t <= 10] = s2[12 <= t] = 0

# add some noise into the mix
nse = 0.01 * np.random.random(size=len(t))

x = s1 + s2 + nse  # the signal
NFFT = 1024  # the length of the windowing segments
Fs = int(1.0 / dt)  # the sampling frequency

fig, (ax1, ax2) = plt.subplots(nrows=2)
ax1.plot(t, x)
Pxx, freqs, bins, im = ax2.specgram(x, NFFT=NFFT, Fs=Fs, noverlap=900)
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the matplotlib.image.AxesImage instance representing the data in the plot
plt.show()

<Figure size 640x480 with 2 Axes>